# COVID-19 Data Analysis Notebook

This notebook analyzes COVID-19 time-series data by:
- Loading CSSEGISandData COVID-19 time-series CSVs
- Grouping data by Country/Region (summing over provinces)
- Calculating daily new cases from cumulative data
- Identifying top 5 countries by total confirmed cases
- Creating visualizations of total cases vs. time and mortality rates

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import io
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8')

## 1. Data Ingestion

Load COVID-19 time-series data from Johns Hopkins CSSEGISandData repository.

In [2]:
# URLs for COVID-19 time series data (as of early 2023)
confirmed_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
deaths_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
recovered_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

print("Loading COVID-19 data from Johns Hopkins repository...")

# Load the data
try:
    confirmed_df = pd.read_csv(confirmed_url)
    deaths_df = pd.read_csv(deaths_url)
    recovered_df = pd.read_csv(recovered_url)
    
    print(f"Confirmed cases data shape: {confirmed_df.shape}")
    print(f"Deaths data shape: {deaths_df.shape}")
    print(f"Recovered data shape: {recovered_df.shape}")
    
except Exception as e:
    print(f"Could not load data from URLs: {e}")
    print("Creating sample data for demonstration...")
    
    # Create sample data for demonstration
    countries = ['US', 'India', 'Brazil', 'France', 'Russia', 'UK', 'Turkey', 'Italy', 'Spain', 'Germany']
    provinces = ['California', 'Maharashtra', 'Sao Paulo', 'Ile-de-France', 'Moscow', 'England', 'Istanbul', 'Lombardy', 'Madrid', 'Bavaria']
    
    # Generate date columns (last 365 days for example)
    start_date = pd.Timestamp.now() - pd.DateOffset(days=365)
    date_cols = pd.date_range(start=start_date, periods=365, freq='D').strftime('%m/%d/%y').tolist()
    date_cols = [f"{date}" for date in date_cols]  # Format to match actual data
    
    # Create sample confirmed cases data
    sample_data = []
    for i, (country, province) in enumerate(zip(countries, provinces)):
        row = {'Province/State': province, 'Country/Region': country, 'Lat': np.random.uniform(-60, 70), 'Long': np.random.uniform(-180, 180)}
        
        # Generate cumulative case numbers
        cumulative_cases = 0
        for date in date_cols:
            daily_new = max(0, np.random.poisson(50 + i*10))  # Higher values for first countries
            cumulative_cases += daily_new
            row[date] = cumulative_cases
        
        sample_data.append(row)
    
    confirmed_df = pd.DataFrame(sample_data)
    deaths_df = confirmed_df.copy()
    recovered_df = confirmed_df.copy()
    
    # Adjust death and recovery values
    for df in [deaths_df, recovered_df]:
        for idx in df.index:
            for col in date_cols:
                original_val = confirmed_df.iloc[idx][col]
                if 'deaths' in str(df).lower() or df.equals(deaths_df):
                    df.at[idx, col] = max(0, int(original_val * np.random.uniform(0.01, 0.03)))
                else:  # recovered
                    df.at[idx, col] = max(0, int(original_val * np.random.uniform(0.8, 0.95)))
    
    print(f"Sample confirmed cases data shape: {confirmed_df.shape}")
    print(f"Sample deaths data shape: {deaths_df.shape}")
    print(f"Sample recovered data shape: {recovered_df.shape}")

Loading COVID-19 data from Johns Hopkins repository...
Confirmed cases data shape: (289, 1147)
Deaths data shape: (289, 1147)
Recovered data shape: (274, 1147)


In [3]:
# Display first few rows of each dataset
print("First 5 rows of confirmed cases data:")
print(confirmed_df.head())
print()

print("First 5 rows of deaths data:")
print(deaths_df.head())
print()

print("First 5 rows of recovered data:")
print(recovered_df.head())
print()

First 5 rows of confirmed cases data:
  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  1/29/20  1/30/20  1/31/20  \
0        0        0        0        0        0        0        0        0   
1        0        0        0        0        0        0        0        0   
2        0        0        0        0        0        0        0        0   
3        0        0        0        0        0        0        0        0   
4        0        0        0        0        0        0        0        0   

   2/1/20  2/2/20  2/3/20  2/4/20  2/5/20  2/6/20  2/7

## 2. Data Preprocessing

Group data by Country/Region and convert to time-series format.

In [4]:
# Transpose the data to convert from wide format to long format
# First, let's look at the date columns in the data
date_columns = [col for col in confirmed_df.columns if '/' in col]
print(f"Number of date columns: {len(date_columns)}")
print(f"First 5 date columns: {date_columns[:5]}")
print(f"Last 5 date columns: {date_columns[-5:]}")
print()

Number of date columns: 1145
First 5 date columns: ['Province/State', 'Country/Region', '1/22/20', '1/23/20', '1/24/20']
Last 5 date columns: ['3/5/23', '3/6/23', '3/7/23', '3/8/23', '3/9/23']



In [5]:
# Group by Country/Region, summing over provinces
confirmed_by_country = confirmed_df.groupby('Country/Region')[date_columns].sum().reset_index()
deaths_by_country = deaths_df.groupby('Country/Region')[date_columns].sum().reset_index()
recovered_by_country = recovered_df.groupby('Country/Region')[date_columns].sum().reset_index()

print(f"Confirmed cases by country shape: {confirmed_by_country.shape}")
print(f"Deaths by country shape: {deaths_by_country.shape}")
print(f"Recovered by country shape: {recovered_by_country.shape}")
print()

print("Top 10 countries by most recent confirmed cases:")
latest_date = date_columns[-1]
top_countries = confirmed_by_country[['Country/Region', latest_date]].sort_values(by=latest_date, ascending=False).head(10)
print(top_countries)
print()

ValueError: cannot insert Country/Region, already exists

## 3. Time-Series Transformation

Convert the data to a proper time-series format for analysis.

In [ ]:
# Transform the data to long format for time series analysis
def transform_to_timeseries(df, measure_type):
    """Transform the wide format data to long format for time series analysis"""
    # Melt the dataframe to convert date columns to rows
    date_cols = [col for col in df.columns if '/' in col]
    
    df_melted = df.melt(id_vars=['Country/Region'], 
                         value_vars=date_cols,
                         var_name='Date', 
                         value_name=f'{measure_type}')
    
    # Convert Date column to datetime
    df_melted['Date'] = pd.to_datetime(df_melted['Date'], format='%m/%d/%y', errors='coerce')
    
    # Sort by Country and Date
    df_melted = df_melted.sort_values(['Country/Region', 'Date']).reset_index(drop=True)
    
    return df_melted

# Transform all datasets
confirmed_ts = transform_to_timeseries(confirmed_by_country, 'Confirmed')
deaths_ts = transform_to_timeseries(deaths_by_country, 'Deaths')
recovered_ts = transform_to_timeseries(recovered_by_country, 'Recovered')

print("Time series transformed successfully!")
print(f"Confirmed time series shape: {confirmed_ts.shape}")
print(f"Deaths time series shape: {deaths_ts.shape}")
print(f"Recovered time series shape: {recovered_ts.shape}")
print()

# Combine all measures into one dataframe
covid_combined = confirmed_ts.merge(deaths_ts, on=['Country/Region', 'Date'])
covid_combined = covid_combined.merge(recovered_ts, on=['Country/Region', 'Date'])

print(f"Combined COVID-19 data shape: {covid_combined.shape}")
print(f"First 10 rows of combined data:")
print(covid_combined.head(10))
print()

## 4. Daily New Cases Calculation

Calculate daily new cases from cumulative data.

In [ ]:
# Calculate daily new cases, deaths, and recovered for each country
def calculate_daily_values(df):
    """Calculate daily values from cumulative data"""
    df_sorted = df.sort_values(['Country/Region', 'Date']).copy()
    
    # Calculate daily values using groupby and diff
    df_sorted['Daily_Confirmed'] = df_sorted.groupby('Country/Region')['Confirmed'].diff().fillna(df_sorted['Confirmed'])
    df_sorted['Daily_Deaths'] = df_sorted.groupby('Country/Region')['Deaths'].diff().fillna(df_sorted['Deaths'])
    df_sorted['Daily_Recovered'] = df_sorted.groupby('Country/Region')['Recovered'].diff().fillna(df_sorted['Recovered'])
    
    # Ensure no negative values (due to data corrections)
    df_sorted['Daily_Confirmed'] = df_sorted['Daily_Confirmed'].clip(lower=0)
    df_sorted['Daily_Deaths'] = df_sorted['Daily_Deaths'].clip(lower=0)
    df_sorted['Daily_Recovered'] = df_sorted['Daily_Recovered'].clip(lower=0)
    
    return df_sorted

# Apply the function to calculate daily values
covid_daily = calculate_daily_values(covid_combined)

print("Daily values calculated!")
print(f"Data shape after daily calculation: {covid_daily.shape}")
print()

# Show an example for a specific country
example_country = covid_daily[covid_daily['Country/Region'] == covid_daily['Country/Region'].iloc[0]].head(10)
print(f"Example for {covid_daily['Country/Region'].iloc[0]}:")
print(example_country[['Date', 'Confirmed', 'Daily_Confirmed', 'Deaths', 'Daily_Deaths']])
print()

## 5. Top 5 Countries Identification

Identify the top 5 countries by total confirmed cases.

In [ ]:
# Identify top 5 countries by total confirmed cases
# Get the most recent date in the dataset
most_recent_date = covid_daily['Date'].max()
print(f"Most recent date in dataset: {most_recent_date}")

# Get the latest data for each country
latest_data = covid_daily[covid_daily['Date'] == most_recent_date].copy()

# Sort by confirmed cases and get top 5
top_5_countries = latest_data.nlargest(5, 'Confirmed')[['Country/Region', 'Confirmed', 'Deaths', 'Recovered']].reset_index(drop=True)
top_5_country_names = top_5_countries['Country/Region'].tolist()

print("Top 5 Countries by Total Confirmed Cases:")
print(top_5_countries)
print()

# Calculate mortality rate for these top 5 countries
top_5_countries['Mortality_Rate'] = (top_5_countries['Deaths'] / top_5_countries['Confirmed'] * 100).round(2)
print("Top 5 Countries with Mortality Rate:")
print(top_5_countries[['Country/Region', 'Confirmed', 'Deaths', 'Mortality_Rate']])
print()

## 6. Time Series Visualization - Total Cases vs. Time

Create plots showing total cases vs. time for top 5 countries.

In [ ]:
# Plot total cases vs. time for top 5 countries
plt.figure(figsize=(14, 8))

for country in top_5_country_names:
    country_data = covid_daily[covid_daily['Country/Region'] == country]
    plt.plot(country_data['Date'], country_data['Confirmed'], label=country, linewidth=2)

plt.title('Total Confirmed Cases Over Time - Top 5 Countries', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Total Confirmed Cases', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
print()

In [ ]:
# Plot daily new cases vs. time for top 5 countries
plt.figure(figsize=(14, 8))

for country in top_5_country_names:
    country_data = covid_daily[covid_daily['Country/Region'] == country]
    plt.plot(country_data['Date'], country_data['Daily_Confirmed'], label=country, linewidth=1.5)

plt.title('Daily New Confirmed Cases Over Time - Top 5 Countries', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Daily New Confirmed Cases', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
print()

## 7. Mortality Rate Visualization

Create plots showing mortality rate (Deaths/Cases) for top countries.

In [ ]:
# Calculate mortality rate over time for each country
covid_daily['Mortality_Rate'] = (covid_daily['Deaths'] / covid_daily['Confirmed'] * 100).round(4)

# Replace infinite values with 0 if division by zero occurred
covid_daily['Mortality_Rate'] = covid_daily['Mortality_Rate'].replace([np.inf, -np.inf], 0)

# Create mortality rate plot for top 5 countries
plt.figure(figsize=(14, 8))

for country in top_5_country_names:
    country_data = covid_daily[covid_daily['Country/Region'] == country]
    plt.plot(country_data['Date'], country_data['Mortality_Rate'], label=country, linewidth=2)

plt.title('Mortality Rate Over Time - Top 5 Countries', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Mortality Rate (%)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
print()

In [ ]:
# Plot mortality rate as a bar chart for the most recent data
latest_mortality = latest_data[latest_data['Country/Region'].isin(top_5_country_names)].copy()
latest_mortality['Mortality_Rate'] = (latest_mortality['Deaths'] / latest_mortality['Confirmed'] * 100).round(2)

plt.figure(figsize=(12, 6))
bars = plt.bar(latest_mortality['Country/Region'], latest_mortality['Mortality_Rate'], 
               color=['red', 'orange', 'yellow', 'green', 'blue'])
plt.title('Current Mortality Rate (%) - Top 5 Countries', fontsize=16)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Mortality Rate (%)', fontsize=12)
plt.xticks(rotation=45)

# Add value labels on bars
for bar, rate in zip(bars, latest_mortality['Mortality_Rate']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{rate}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()
print()

## 8. Advanced Analysis

Perform additional analysis on the COVID-19 data.

In [ ]:
# Calculate case fatality rate (CFR) for all countries
latest_global = covid_daily[covid_daily['Date'] == most_recent_date].copy()
latest_global['CFR'] = (latest_global['Deaths'] / latest_global['Confirmed'] * 100).round(2)

# Filter out countries with very low case numbers to avoid noise
significant_countries = latest_global[latest_global['Confirmed'] >= 1000].copy()
significant_countries = significant_countries[significant_countries['CFR'] >= 0]  # Remove negative rates

# Get top 10 countries by case fatality rate
top_cfr = significant_countries.nlargest(10, 'CFR')[['Country/Region', 'Confirmed', 'Deaths', 'CFR']].reset_index(drop=True)

print("Top 10 Countries by Case Fatality Rate (with >=1000 confirmed cases):")
print(top_cfr)
print()

# Get bottom 10 countries by case fatality rate
bottom_cfr = significant_countries.nsmallest(10, 'CFR')[['Country/Region', 'Confirmed', 'Deaths', 'CFR']].reset_index(drop=True)

print("Bottom 10 Countries by Case Fatality Rate (with >=1000 confirmed cases):")
print(bottom_cfr)
print()

In [ ]:
# Visualize the case fatality rate comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Top 10 CFR
axes[0].barh(top_cfr['Country/Region'], top_cfr['CFR'], color='lightcoral')
axes[0].set_title('Top 10 Countries by Case Fatality Rate')
axes[0].set_xlabel('Case Fatality Rate (%)')

# Bottom 10 CFR
axes[1].barh(bottom_cfr['Country/Region'], bottom_cfr['CFR'], color='lightgreen')
axes[1].set_title('Bottom 10 Countries by Case Fatality Rate')
axes[1].set_xlabel('Case Fatality Rate (%)')

plt.tight_layout()
plt.show()
print()

## 9. Summary and Insights

Summarize key findings and insights from the COVID-19 analysis.

In [ ]:
print("=== COVID-19 PANDEMIC ANALYSIS SUMMARY ===\n")

print("Dataset Overview:")
print(f"- Time span: {covid_daily['Date'].min()} to {covid_daily['Date'].max()}")
print(f"- Number of countries analyzed: {covid_daily['Country/Region'].nunique()}")
print(f"- Total confirmed cases (latest): {latest_data['Confirmed'].sum():,}")
print(f"- Total deaths (latest): {latest_data['Deaths'].sum():,}")
print()

print("Top 5 Countries by Total Confirmed Cases:")
for idx, row in top_5_countries.iterrows():
    print(f"  {idx+1}. {row['Country/Region']}: {row['Confirmed']:,} confirmed cases")
print()

print("Key Insights:")
print(f"1. The pandemic has affected {covid_daily['Country/Region'].nunique()} countries/regions globally")
print(f"2. Total confirmed cases reached {latest_data['Confirmed'].sum():,} worldwide")
print(f"3. Total deaths reached {latest_data['Deaths'].sum():,} worldwide")
print(f"4. Average global mortality rate: {(latest_data['Deaths'].sum() / latest_data['Confirmed'].sum() * 100):.2f}%")
print()

print("Methodology:")
print("- Data was grouped by Country/Region (summing over provinces/states)")
print("- Daily new cases were calculated from cumulative data using differencing")
print("- Mortality rate was calculated as (Deaths / Confirmed) * 100")
print("- Time series visualizations show trends over the pandemic timeline")
print()

print("Data Quality Notes:")
print("- Data was aggregated at the country level to provide a clearer picture")
print("- Negative daily values were clipped to zero (representing data corrections)")
print("- Countries with very low case numbers were filtered for mortality rate analysis")
print()

print("This analysis demonstrates the power of pandas for time-series analysis")
print("and the importance of proper data preprocessing for meaningful insights.")